In [ ]:
# Indices

import pandas as pd
import numpy as np
from utils.quilt import load_datasets

year = range(2000,2035)

fuel = [     
    "sähkö",
    "lämpö ja kylmä",
    "maakaasu",
    "nestekaasu",
    "lämmitysöljy",
    "diesel",
    "bensiini",
    "ruskohiili",
    "hiili",
    "muut fossiiliset (sis. turve)",
    "kasviöljy",
    "biopolttoaine",
    "muu biomassa",
    "aurinkoterminen",
    "maalämpö",
    "tuulivoima",
    "vesivoima",
    "aurinkosähkö"
]

emission = [
    "hiilidioksidi",
    "metaani",
    "typpidioksidi",
    "typpioksiduuli",
    "hiilimonoksidi",
    "hiukkaset",
    "rikkidioksidi",
    "typenoksidit",
    "fluorihiilivedyt",
    "perfluorihiilivedyt",
    "rikkiheksafluoridi",
    "typpitrifluoridi",
    "F-kaasut",
    "pienhiukkaspäästö",
    "kasvihuonekaasupäästö"
]

sector = [
    "rakennukset",
    "kunnalliset rakennukset, laitteistot/tilat",
    "ei-kunnalliset rakennukset, laitteistot/tilat",
    "asuinrakennukset",
    "julkinen valaistus",
    "teollisuus, ei-ETS",
    "teollisuus, ETS (ei suositella)",
    "kuljetus",
    "kunnalliskalusto",
    "julkinen liikenne",
    "yksityinen ja kaupallinen liikenne",
    "energia",
    "paikallinen sähköntuotanto",
    "paikallinen lämmön ja kylmän tuotanto",
    "vesi",
    "jätteet",
    "maankäytön suunnittelu",
    "maatalous ja metsänhoito (sis. kalanjalostamot)",
    "ympäristö ja biodiversiteetti",
    "terveys",
    "pelastuspalvelut ja hätäpalvelut",
    "turismi",
    "muu sektori",
    "sopeutuminen"
]

subsector_of = [0,0,0,0,0,0,0,7,7,7,7,11,11,11,14,15,16,17,18,19,20,21,22,23]

treatment = ['solid waste disposal', 'composting', 'digestion', 'incineration', 'waste water treatment']

source = ['in - in', 'in - out', 'out - in']

#df.columns from Statistics Finland
"""Index(['Rakennuksen lämmitysaine', 'Alue', 'Vuosi',
       'Rakennuksen kerrosala (m2) Yhteensä',
       'Rakennuksen kerrosala (m2) Erilliset pientalot',
       'Rakennuksen kerrosala (m2) Rivi- ja ketjutalot',
       'Rakennuksen kerrosala (m2) Asuinkerrostalot',
       'Rakennuksen kerrosala (m2) Liikerakennukset',
       'Rakennuksen kerrosala (m2) Toimistorakennukset',
       'Rakennuksen kerrosala (m2) Liikenteen rakennukset',
       'Rakennuksen kerrosala (m2) Hoitoalan rakennukset',
       'Rakennuksen kerrosala (m2) Kokoontumisrakennukset',
       'Rakennuksen kerrosala (m2) Opetusrakennukset',
       'Rakennuksen kerrosala (m2) Teollisuusrakennukset',
       'Rakennuksen kerrosala (m2) Varastorakennukset',
       'Rakennuksen kerrosala (m2) Muut rakennukset'],
      dtype='object')
"""

# From Statistics Finland
building = ['detached houses','row houses','apartment buildings',
              'commercial buildings','office buildings','transport buildings','health care buildings',
              'congregation buildings','educational buildings','industrial buildings','storage buildings',
              'other buildings'
    ]

# From SECAP reporting
building_short = ['residential', 'commercial and institutional', 'industry and construction', 'energy industries']

def buildings():
    df = pd.DataFrame({
        'building':building,
        'building_short':['residential'] * 3 + ['commercial and institutional'] * 6 + ['industry and construction'] * 2 +
        ['commercial and institutional'],
        'result':1
    }).set_index(['building','building_short'])
    return df

final_use = ['consumer electricity','district heating','gas', 'geothermal', 'heating electricity',
            'industry and machinery','oil heating', 'other heating','road transport','coal','wood heating']

def scopes_of_uses():
    df = pd.DataFrame({
        'final_use':final_use,
        'scope':['2','2','2','2','2','1','1','1','1','1','1'],
        'result':1
    }).set_index(['final_use','scope'])
    return df

energy_class = ['old', 'new', 'low-energy']

mode = ['walking', 'cycling', 'private car', 'bus', 'truck', 'van', 'tram', 'metro', 'train', 'flight', 'boat', 'machinery']

mode_short = ['road','rail','marine','aviation','off-road']

def modes():
    df = pd.DataFrame({
        'mode':mode,
        'mode_short':['road']*6 + ['rail']*3 + ['marine'] + ['aviation'] + ['off-road'],
        'result':1
    }).set_index(['mode','mode_short'])
    return df

animal = ['cow', 'pig', 'sheep']

landuse = ['forest', 'field', 'turf field', 'peatland']

scope = ['1','2','3']

gas = ['CO2e', 'CO2', 'CH4'] # Add PM2.5 and other gases relevant for other impacts

def municipality():
    df = pd.DataFrame({
        'municipality':['Tampere'],
        'result':1
    }).set_index(['municipality'])
    return df

# https://ghgprotocol.org/sites/default/files/standards/GHGP_GPC_0.pdf Figure 2
def gpc():
    tmp = ['gpc','basic','scope','name']
    df = pd.read_csv("gpc_classification.csv")[tmp].set_index(tmp)
    df['result'] = 1
    return df

def global_warming_potential():
    df = pd.DataFrame({'gas':gas, 'result':[1,1,10]})
    return df.set_index('gas')

# A test function to replace groupby for making sure that Iter column is not dropped even if it is not explicitly mentioned.
# This can be used if we end up using probabilistic variables.
# df is the variable dataframe and keep is a list of index column names to keep in the outcome.

def grupby(df, keep):
    out = list(set(df.columns) & set(keep+['Iter']))
    out = df.groupby(out)
    return out

def fillna(object, cols):
    ind = object.index.names
    out = object.reset_index()
    if cols != [None]:
        for i in list(cols):
            a1 = out[out[i].notna()]
            a2 = out[out[i].isna()].drop(i, axis=1).assign(tmp=1)
            addition = pd.DataFrame({i:pd.unique(a1[i]), 'tmp':1})
            a2 = a2.merge(addition).drop('tmp',axis=1)
            out = a1.append(a2)
    if ind != [None]:
        out = out.set_index(ind)
    return out

#display(grupby(ghg_emissions_from_heating_of_stationary_sources().assign(Iter = 1), ['scope']).sum().reset_index())
display(scopes_of_uses())

In [ ]:
#tst = load_datasets("jtuomsto/cnh/power_consumption_example")

# datasets from quilt/jyrjola
#
# aluesarjat: Helsingin seudun väestö-, rakennus- ja tulotilastoja 10 tiedostoa
# tst = load_datasets('jyrjola/energiaatlas/buildings') # rakennuskantarekisteri? 77267 rows × 109 columns
# energiateollisuus: kaukolämmön ja sähkön tuotannon tilastoja ja polttoaineen kulutus
#tst = load_datasets('jyrjola/energiateollisuus/electricity_production_hourly') # 78888 rows × 11 columns, 2010-2018
# fingrid: sähkönkulutus ja siirto 2 tiedostoa, esim. 3 min välein
# fingrid_hourly: sähkönkulutus ja hinta, 3 tiedostoa
# fingrid_realtime: 1 tiedosto power, 3/2019
# fmi: 3 tiedostoa: heating_degree_days, solar_radiation_kumpula, solar_radiation savilahti
# hsy: 2 tiedostoa: buildings, pks_khk_paastot
# karttahel: 1 tiedosto: buildings
# lipasto: 3 tiedostoa 
#├─car_unit_emissions # vain henkilöautot
#├─emissions_by_municipality
#└─mileage_per_engine_type
# osm:
#└─helsinki_bike_lane
#statfi
#└─fuel_classification # 84 rows × 10 columns, co2e_emission_factor, calorific_value
#traficom: 11 tiedostoa:
#├─cars_by_engine_type
#├─tf010_kanta_tau_101
#├─tf020_ensirek_tau_102
#├─vehicle_register_2014q4
#├─vehicle_register_2015q4
#├─vehicle_register_2017q3
#├─vehicle_register_2018q1
#├─vehicle_register_2018q2
#├─vehicle_register_2018q3
#├─vehicle_register_2019q1
#└─...
#ymparistotilastot: 19 tiedostoa koskien Helsinkiä/pääkaupunkiseutua?
#├─e03_energian_kokonaiskulutus
#├─e06_ymparistopenniasiakkaat
#├─e12_helsingin_kaukolammon_sahkonkulutus
#├─e13_helen_vantaa_kaukolammon_sahkonhankinta
#├─e14_helen_ominaispaastot
#├─e15_helen_kaukolampo_jaahdytys
#├─e1_sahkonkulutus
#├─e21_kaup_rakennus_tyypeittain
#├─e23_energian_kuluttajaryhmat_kokonaiskulutus
#├─e25_kl_polttoaineet_pks
#└─...

#tst = load_datasets('jyrjola/lipasto/car_unit_emissions')
#display(tst)
#pd.unique(tst['Engine'])

In [ ]:
##### THIS CELL IS NOT USED

##### Get building data from Statistics Finland

#df = pd.read_csv('http://pxnet2.stat.fi/PXWeb/sq/930e8afc-c473-4288-a656-a6d0478ca5d7')

#df = pd.read_csv('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeCsvWithHeadingAndSemiColon',
#     sep=";", skiprows=2)

#df = pd.read_json('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeJsonStat')

#df = pd.read_excel('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeExcelX',
#                  header=True, sheet_name='006_116h_2019')

import os
import csv
import datetime
import urllib.request
import urllib.parse
import urllib.error
import zlib
import time

"""
from pandas_pcaxis.pxweb_api import PXWebAPI
api = PXWebAPI('http://pxnet2.stat.fi/PXWeb', 'fi')
print(api.list_databases())
print(api.list_topics('StatFin/asu/rakke'))
px_file = api.get_table('StatFin/asu/rakke/statfin_rakke_pxt_116g.px')
df = px_file.to_df()

"""
#import px_reader
#px_obj = px_reader.Px('a_px_file_on_filesystem.px')
#pandas_dataframe = px_obj.pd_dataframe()

def download_px(px_objs, target_dir='.', compressed=False, sleep=1, refresh='check'):
    """
    Fetch PC Axis files for given list of Px objects
    Save the files to target directory

    WARNING: Statfin database contains over 2500 PX files with many gigabytes of data.
    """

    refresh_options = ['never', 'check', 'always']
    if refresh not in refresh_options:
        raise ValueError('Invalid value for refresh, must be one of "{}"'.format(
            '", "'.join(refresh_options)))

    if not isinstance(px_objs, list):
        px_objs = [px_objs]

    for px_obj in px_objs:
        url_parts = urllib.parse.urlparse(px_obj.path)
        # url_parts.path starts with '/'
        target_path = os.path.join(target_dir, url_parts.path[1:])
        target_path = os.path.abspath(target_path)

        if refresh != "always" and os.path.exists(target_path):
            if refresh == 'check':
                if is_latest(px_obj.path, target_path):
                    print('File {} is already latest, skipping'.format(target_path))
                    time.sleep(1)
                    continue
            elif refresh == 'never':
                print('File {} already exists, skipping'.format(target_path))
                continue

        print('Downloading file from {} ...'.format(px_obj.path), end=' ')
        try:
            request = urllib.request.Request(px_obj.path)
            if compressed:
                request.add_header('Accept-encoding', 'gzip')
            response = urllib.request.urlopen(request)
        except urllib.error.HTTPError as e:
            print('ERROR:', e)
            print('Response headers:', e.headers)
            time.sleep(sleep)
            continue

        makedirs(target_path)
        try:
            with open(target_path, 'wb') as f:
                data = response.read()
                if compressed:
                    data = zlib.decompress(data, zlib.MAX_WBITS | 16)
                f.write(data)
        except IOError as e:
            print('ERROR:', e)
            time.sleep(sleep)
            continue

        print('done')
        time.sleep(sleep)

#df = download_px('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/')

#import statsfi_px_api
#import px_reader
#px_info = statsfi_px_api.list_available_px(url="http://pxweb2.stat.fi/database/StatFin/StatFin_rap.csv")

In [ ]:
### Data from statfi, manually


In [ ]:
################ Heating of buildings

## Energy use

def floor_area_of_buildings():
    #'http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?downloadfile=FileTypeCsvWithHeadingAndSemiColon'

    df = pd.read_csv('006_116h_2019.csv', skiprows=2, encoding = 'iso8859_10')

    df.columns = ['final_use','municipality','year','total'] + building
    df = df.drop('total', axis=1)
    df = df[df['final_use'] != 'Yhteensä']
    df = df[df['municipality'] != 'KOKO MAA']
    df['year'] = list(int(x) for x in df['year'])
    
    df.replace({
        'Kauko- tai aluelämpö':'district heating',
        'Öljy, kaasu':'oil heating',
        'Sähkö':'heating electricity',
        'Kivihiili':'coal',
        'Puu, turve':'wood heating',
        'Maalämpö':'geothermal',
        'Muu, tuntematon':'other heating'}, inplace=True)

    df = pd.melt(df, id_vars=['final_use','municipality','year'], var_name='building', value_name='result')

    df = df.set_index(['final_use','municipality','year','building'])

    return df

def fractions_of_energy_classes(): # fraction of buildings belonging to different efficiency groups
    #'http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116g.px/table/tableViewLayout2/?downloadfile=FileTypeCsvWithHeadingAndComma'
    # http://pxnet2.stat.fi/PXWeb/sq/776add46-5141-4e27-860b-cac210d79c50

    df = pd.read_csv('006_116g_2019.csv', skiprows=2, encoding = 'iso8859_10')

    df.columns = ['built','municipality','year'] + building
    df = df.merge(pd.DataFrame({
        'built':['- 1920', '1921 - 1939', '1940 - 1959', '1960 - 1969',
       '1970 - 1979', '1980 - 1989', '1990 - 1999', '2000 - 2009',
       '2010 -', 'Tuntematon'],
        'energy_class':['old']*7 + ['new']*2 + ['old']
    }))
    df['year'] = list(int(x) for x in df['year'])
    
    df = pd.melt(df, id_vars=['built','energy_class','municipality','year'], var_name='building', value_name='result')
    df1 = df.groupby(['energy_class','municipality','year','building']).sum()
    df2 = df.groupby(['municipality','year','building']).sum()
    df = df1 / df2

    return df

def energy_consumption_of_buildings():
    # http://en.opasnet.org/w/Energy_use_of_buildings#Energy_efficiency_in_heating
    df = pd.DataFrame({
        'energy_class':['old','new','low-energy'],
        'result':[150,70,35] # kWh/m²/a
    })
    return df.set_index(['energy_class'])

def energy_use_from_heating_of_buildings(
    p1 = municipality(),
    p2 = floor_area_of_buildings(),
    p3 = fractions_of_energy_classes(),
    p4 = energy_consumption_of_buildings()):

    out = (p1 * p2).dropna()
    out = (out * p3).dropna() * p4 * 1E-6 # kWh/a --> GWh/a
    
    out = out.groupby(list(set(out.index.names) - {'energy_class'})).sum()
    return out

def emission_factors_of_energy(p1 = scopes_of_uses()): # kton/GWh # ASSUME NO CHANGE SINCE 2018
    out = pd.read_csv('emission_factors_of_energy_consumption.csv')
    out = out.drop(['emission','energy_use','level5'], axis=1).rename(columns={'emission_factor':'result'})
    out = out.append(out[out['year']==2018].assign(year=2019))
    out = out.set_index(['final_use','year','municipality','emission_trade','gas'])
    out = out * p1
    out = fillna(out, ['municipality'])
    return out

def loss_from_the_grid(): # Loss in the system, given as a ratio to the FINAL energy consumption
    df = pd.DataFrame({
        'final_use':final_use,
        'scope':'3',
        'result':0.1}) # % # ALSO CONTAINS SYSTEM INEFFICIENCIES (hyötysuhde)
    #BUT THIS DEPENDS ON Production type
    return df.set_index(['final_use','scope'])

def emissions_from_heating_of_stationary_sources(
    p1 = emission_factors_of_energy(),
    p2 = energy_use_from_heating_of_buildings(),
    p3 = loss_from_the_grid()):

    out = (p1 * p2).dropna() # kton

    loss = (out.reset_index('scope').drop('scope',1) * p3).dropna()

    out = pd.concat([out.reset_index(), loss.reset_index()])
    out['year'] = list(None if pd.isnull(x) else int(x) for x in out['year'])
    out= out.groupby(list(set(out.columns) - {'emission_trade','result'})).sum()

    return out

def ghg_emissions_from_heating_of_stationary_sources(
    p1 = emissions_from_heating_of_stationary_sources(),
    p2 = global_warming_potential()):

    out = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    out = out.groupby(list(set(out.index.names) - {'gas','result'})).sum()

    return out

display(ghg_emissions_from_heating_of_stationary_sources())

In [ ]:
######### Consumer electricity

def specific_electricity_consumption():
    df = pd.DataFrame({
        'building':building,
        'result':150 # kWh/m²/a
    })
    return df.set_index('building')

def energy_use_from_consumer_electricity(
    p1 = municipality(),
    p2 = floor_area_of_buildings(),
    p3 = specific_electricity_consumption()):
    
    out = (p1 * p2).dropna().reset_index()
    out['final_use'] = 'consumer electricity'
    out = out.groupby(list(set(out.columns) - {'result'})).sum()
    out = out * p3 * 1E-6 # kWh/a --> GWh/a

    return out

def emissions_from_consumer_electricity(
    p1 = energy_use_from_consumer_electricity(),
    p2 = emission_factors_of_energy(),
    p3 = loss_from_the_grid()):

    out = p2.reset_index('emission_trade').drop('emission_trade', axis=1) # always NaN for consumer electricity
    out = (p1 * out).dropna() # ton/a

    loss = (out.reset_index('scope').drop('scope',1) * p3).dropna()

    out = pd.concat([out.reset_index(), loss.reset_index()])#.dropna() # non-electricity final uses dropped
    out['year'] = list(None if pd.isnull(x) else int(x) for x in out['year'])
    out = out.groupby(list(set(out.columns) - {'result'})).sum()
    
    return out

def ghg_emissions_from_consumer_electricity(
    p1 = emissions_from_consumer_electricity(),
    p2 = global_warming_potential()):

    out = (p1 * p2).dropna()  * 1E-3 # ton/a --> kton/a (CO2e)
    out = out.groupby(list(set(out.index.names) - {'gas', 'result'})).sum()

    return out

display(ghg_emissions_from_consumer_electricity())

In [ ]:
# Missing stationary sources:

# I.5.X Agriculture, forestry, and fishing activities
# I.6.X Non-specified sources
# I.7.X Fugitive emissions from mining, processing, storage, and transportation of coal
# I.8.X Fugitive emissions from oil and natural gas systems

In [ ]:
################# Transport

def transport_activity_of_people():
    df = pd.DataFrame({
        'transboundary':source,
        'result':[290,50,10] # trips per day
    })
    
    return df.set_index(['transboundary'])

def average_trip_length():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # km/trip
    })
    
    return df.set_index(['transboundary','mode'])

def modal_share_of_transport():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # fraction
    })
    
    return df.set_index(['transboundary','mode'])

def energy_intensity_of_transport_modes():
    df = pd.DataFrame({
        'mode':mode,
        'fuel':['electricity', 'electricity', 'gasoline', 'diesel', 'diesel', 'diesel',
                'electricity', 'electricity', 'electricity','kerosene','light oil','diesel'],
        'result':range(len(mode)) # MJ/km
    })
    
    return df.set_index(['mode','fuel'])

def emission_factors_of_transport_modes():
    df = pd.DataFrame({
        'fuel':['electricity', 'gasoline', 'diesel','kerosene','light oil']*len(gas),
        'scope':['2','1','1','1','1']*len(gas),
        'gas':np.repeat(gas,5),
        'result':5 # g/MJ = ton/TJ
    })
    
    return df.set_index(['fuel','scope','gas'])

def energy_use_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = transport_activity_of_people(),
    p2 = modal_share_of_transport(),
    p3 = energy_intensity_of_transport_modes()):
    
    df = p1 * p2 * p3 * 1E-6 * 365.25 # MJ/d --> TJ/a
    df = df.groupby(['transboundary','fuel','mode']).sum()
    
    return df

def emissions_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = energy_use_from_transportation(),
    p2 = emission_factors_of_transport_modes()):
    
    df = p1 * p2 # ton/a
    df = df.groupby(['transboundary','fuel','scope','gas','mode']).sum()
    
    return df

def ghg_emissions_from_transportation(
    p1 = emissions_from_transportation(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['transboundary','fuel','scope','mode']).sum()
    
    return df

display(ghg_emissions_from_transportation())

In [ ]:
####### Waste

def amount_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(source),
        'source':np.repeat(source, len(treatment)),
        'result':1000 # ton/a
    })
    return df.set_index(['treatment','source'])

def emission_factors_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(gas),
        'gas':np.repeat(gas, len(treatment)),
        'result':10 # kg/ton
    })
    return df.set_index(['treatment','gas'])

def emissions_from_waste(
    p1 = amount_of_waste(),
    p2 = emission_factors_of_waste()):
    
    df = p1 * p2 * 1E-3 # kg/a --> ton/a
    df = df.groupby(['treatment','gas','source']).sum() # redundant because there are currently no more indices
    
    return df

def ghg_emissions_from_waste(
    p1 = emissions_from_waste(),
    p2 = global_warming_potential()):
 
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['treatment','source']).sum()
    
    return df

display(ghg_emissions_from_waste())

In [ ]:
################## AFOLU

def number_of_farm_animals():
    df = pd.DataFrame({
        'animal':animal,
        'result':2000 # number
    })
    
    return df.set_index('animal')

def land_area():
    df = pd.DataFrame({
        'landuse':landuse,
        'result':5000 # ha
    })
    
    return df.set_index('landuse')

def emission_factors_of_afolu():
    df = pd.DataFrame({
        'source':['livestock']*len(animal)+['land']*len(landuse)+['aggregate']*len(landuse),
        'animal':animal+[None]*len(landuse)*2,
        'landuse':[None]*len(animal)+landuse*2,
        'gas':['CO2']*(len(animal)+len(landuse))+['CH4']*len(landuse),
        'result':1 # g/animal/a or g/ha/a
    })
    return df.set_index(['source','animal','landuse','gas'])

def emissions_from_afolu(
    p1 = number_of_farm_animals(),
    p2 = land_area(),
    p3 = emission_factors_of_afolu()):
    
    df = p1 * p3 * 1E-6 # g/a --> ton/a
    df = df.groupby(['source','gas']).sum()
    
    df2 = p2 * p3 * 1E-6 # g/a --> ton/a
    df2 = df2.groupby(['source','gas']).sum()
    
    df = pd.concat([df, df2])   
    
    return df

def ghg_emissions_from_afolu(
    p1 = emissions_from_afolu(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['source']).sum()
    
    return df

#display(ghg_emissions_from_afolu())

def why_does_field_appear_in_livestock( ## WTF?!?
    p1 = land_area(),
    p2 = emission_factors_of_afolu()):
    
    df = p1 * p2
    return df

display(emission_factors_of_afolu())
display(land_area())
display(why_does_field_appear_in_livestock())


In [ ]:
############## IPPU

def emissions_from_ippu():
    df = pd.DataFrame({
        'source':['industrial processes','product use']*len(gas),
        'gas':np.repeat(gas,2),
        'result':100 # ton/a
    })
    
    return df.set_index(['source','gas'])

def ghg_emissions_from_ippu(
    p1 = emissions_from_ippu(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    
    return df

display(ghg_emissions_from_ippu())

In [ ]:
##################### Other scope 3

def ghg_emissions_from_other_scope3():
    df = pd.DataFrame({
        'source':['other scope3'],
        'result':20 # kton/a (CO2e)
    })
    
    return df.set_index(['source'])

display(ghg_emissions_from_other_scope3())

In [ ]:
##### Total GHG emissions from the municipality using GPC classification (compatible with basic/basic+ classification)
    
# NOTE! Assumes that no time dimension is used

def total_ghg_emissions_from_the_municipality( # kton/a (CO2e)
    p1 = ghg_emissions_from_heating_of_stationary_sources(),
    p2 = ghg_emissions_from_consumer_electricity(),
    p3 = ghg_emissions_from_transportation(),
    p4 = ghg_emissions_from_waste(),
    p5 = ghg_emissions_from_ippu(),
    p6 = ghg_emissions_from_afolu(),
    p7 = ghg_emissions_from_other_scope3(),
    c1 = buildings(),
    c2 = gpc()):
    
    indices = ['gpc','municipality','year']
    
    tmp = pd.DataFrame({
        'scope':['1','2','3']*len(building_short),
        'building_short':np.repeat(building_short, 3),
        'gpc':['I.1.1','I.1.2','I.1.3','I.2.1','I.2.2','I.2.3','I.3.1','I.3.2','I.3.3','I.4.1','I.4.2','I.4.3'],
        'result':1
    }).set_index(['scope','building_short','gpc'])
    df = (pd.concat([p1, p2]) * c1 * tmp).dropna()
    df = df.groupby(indices).sum()
    
    tmp = p3 * pd.DataFrame({ # Half of the emission is allocated to starting point and half to destination
        'transboundary':source,
        'municipality':'Kunta', ####
        'year':2010,  ########
        'result':[1, 0.5, 0.5]
    }).set_index(['transboundary','municipality','year'])
    
    tmp = tmp * modes() * pd.DataFrame({
        'scope':scope*len(mode_short),
        'mode_short':np.repeat(mode_short, len(scope)),
        'gpc':['II.1.1','II.1.2','II.1.3','II.2.1','II.2.2','II.2.3','II.3.1','II.3.2','II.3.3',
               'II.4.1','II.4.2','II.4.3','II.5.1','II.5.2','II.5.3'],
        'result':1
    }).set_index(['scope','mode_short','gpc'])
    
    tmp = tmp.groupby(indices).sum() # kton/a (CO2e)

    df = pd.concat([df, tmp])

    tmp = p4 * pd.DataFrame({
        'treatment':np.repeat(treatment, len(source)),
        'source':source*len(treatment),
        'gpc':['III.1.1','III.1.2','III.1.3','III.2.1','III.2.2','III.2.3','III.2.1','III.2.2','III.2.3',
               'III.3.1','III.3.2','III.3.3','III.4.1','III.4.2','III.4.3'],
        'municipality':'Kunta', ####
        'year':2010,  ########
        'result':1
    }).set_index(['treatment','source','gpc','municipality','year'])
    tmp = tmp.groupby(indices).sum()

    df = pd.concat([df, tmp])
    
    tmp = p5 * pd.DataFrame({
        'source':['industrial processes','product use'],
        'gpc':['IV.1','IV.2'],
        'municipality':'Kunta', ####
        'year':2010,  ########
        'result':1
    }).set_index(['source','gpc','municipality','year'])
    
    tmp = tmp.groupby(indices).sum()

    df = pd.concat([df, tmp])
    
    tmp = p6 * pd.DataFrame({
        'source':['livestock','land','aggregate'],
        'gpc':['V.1','V.2','V.3'],
        'municipality':'Kunta', ####
        'year':2010,  ########
        'result':1
    }).set_index(['source','gpc','municipality','year'])
    
    tmp = tmp.groupby(indices).sum()

    df = pd.concat([df, tmp])
    
    tmp = p7 * pd.DataFrame({
        'source':['other scope3'],
        'gpc':['VI.1'],
        'municipality':'Kunta', ####
        'year':2010,  ########
        'result':1
    }).set_index(['source','gpc','municipality','year'])
    tmp = tmp.groupby(indices).sum()

    df = pd.concat([df, tmp])
    df = df * c2
    
    return df

display(total_ghg_emissions_from_the_municipality())

In [ ]:
####### Total energy use and GHG emissions using SECAP reporting

### NOTE! The end result should be more specific to be useful for SECAP

def energy_use_of_the_municipality( # GWh/a
    p1 = energy_use_from_heating_of_buildings().reset_index(),
    p2 = energy_use_from_consumer_electricity().reset_index(),
    p3 = energy_use_from_transportation().reset_index()):
    
    p1['sector'] = 'heating'
    p2['sector'] = 'electricity'
    p3['sector'] = 'transportation'
    
    df = pd.concat([p1, p2])
    p3['final_use'] = p3['fuel']
    p3['result'] / 3.6 # TJ/a --> GWh/a
    df = pd.concat([df[['sector','final_use','result']], p3[['sector','final_use','result']]])
    df = df.groupby(['sector','final_use']).sum()
    
    return df

display(energy_use_of_the_municipality())

In [ ]:
#class MyClass(object): pass

#objs = [MyClass() for i in range(50)]

#display(objs)

class MyClass(object):
    def __init__(self, number):
        self.number = number

my_objects = []

for i in range(100):
    my_objects.append(MyClass(i))

# later

#for obj in my_objects:
#    print(obj)